In [11]:
from langchain.llms import GooglePalm

In [12]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import requests
from bs4 import BeautifulSoup

In [13]:
import pandas as pd

In [14]:
with open("google_key.txt", "r") as f:
    api_key=f.read()

In [15]:
llm=GooglePalm(google_api_key=api_key, temperature=0.7)

In [16]:
urls=["https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html",
     "https://timesofindia.indiatimes.com/city/kochi/centre-calls-health-meet-as-covid-rises-in-kerala-10-deaths-this-month/articleshow/106106851.cms"]
responses=[]

for url in urls:
    response=requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        responses.append((text, response.url))
        
    else:
        print(f"Failed to retrieve the URL. Url={url} Status_code :{response.status_code}")

In [17]:
len(responses)

2

In [18]:
import string 
punct=string.punctuation
seps=[]
for i in punct:
    seps.append(i)

In [19]:
text_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "?","." ]+seps+[" "],
    chunk_size=1000,
    chunk_overlap=200
)

In [20]:
splitted_tuple=[]
for i in responses:
    splitted_text= text_splitter.split_text(i[0])
    for j in splitted_text:
        splitted_tuple.append((j, i[1]))

In [21]:
data_dict=[{"chunks": content, "url": url} for content, url in splitted_tuple]

In [22]:
df = pd.DataFrame(data_dict)

In [23]:
def change(data):
    cleaned_data = data.strip()

# Remove extra spaces and newlines between words
    #cleaned_data = ' '.join(cleaned_data.split())

    print(cleaned_data)

In [24]:
change("""gainers losers   top gainers top losers    hdfc bank share price 16562019     tata steel share price 1297421     ntpc share price 29805379     tata motors share price 70495333     hcl technologies share price 14399324     mahindra mahindra share price 164825304       active stocks wed dec 20 2023 155958     tata steel share price  1297  421      state bank of india share price  63615  297      power grid corporation of india share price  2272  286      ntpc share price  29805  379      tata motors share price  70495  333   



 business news news                         indiaÂ  uttarkashi tunnel collapse ndrf team enters tunnel to save 41 trapped workers with ambulances on standby 10 updates 







back




share via








 




uttarkashi tunnel collapse ndrf team enters tunnel to save 41 trapped workers with ambulances on standby 10 updates


3 min read
23 nov 2023 0743 am ist

join us 


livemint""")

gainers losers   top gainers top losers    hdfc bank share price 16562019     tata steel share price 1297421     ntpc share price 29805379     tata motors share price 70495333     hcl technologies share price 14399324     mahindra mahindra share price 164825304       active stocks wed dec 20 2023 155958     tata steel share price  1297  421      state bank of india share price  63615  297      power grid corporation of india share price  2272  286      ntpc share price  29805  379      tata motors share price  70495  333   



 business news news                         indiaÂ  uttarkashi tunnel collapse ndrf team enters tunnel to save 41 trapped workers with ambulances on standby 10 updates 







back




share via








 




uttarkashi tunnel collapse ndrf team enters tunnel to save 41 trapped workers with ambulances on standby 10 updates


3 min read
23 nov 2023 0743 am ist

join us 


livemint


In [25]:
def filter_chunks(text):
    text=text.lower().strip()
    text=text.split()
    words=[]
    for i in text:
        if i not in seps:
            word="".join([j for j in i if j not in seps])
            words.append(word)
    sentence=" ".join(words)
    if(len(sentence)>0):
        return sentence  

In [26]:
filter_chunks("add! .")

'add'

In [27]:
df["chunks"]=df["chunks"].apply(filter_chunks)

In [28]:
df.dropna()

,chunks,url
0,uttarkashi tunnel collapse ndrf team enters tu...,https://www.livemint.com/news/india/uttarkashi...
1,explore sign in epaper subscribe tuesday 2 jan...,https://www.livemint.com/news/india/uttarkashi...
2,top sections news india news world news econom...,https://www.livemint.com/news/india/uttarkashi...
3,a opinion markets stock markets commodity news...,https://www.livemint.com/news/india/uttarkashi...
4,multimedia collections videos webstories photo...,https://www.livemint.com/news/india/uttarkashi...
6,gainers losers top gainers top losers sun phar...,https://www.livemint.com/news/india/uttarkashi...
7,business news news india uttarkashi tunnel col...,https://www.livemint.com/news/india/uttarkashi...
8,a 15member ndrf team led by a commandant has b...,https://www.livemint.com/news/india/uttarkashi...
9,2 medical professionals have been stationed wi...,https://www.livemint.com/news/india/uttarkashi...
10,3 the dedicated freight corridor corporation o...,https://www.livemint.com/news/india/uttarkashi...


In [30]:
csv_path='dataset.csv'
if os.path.exists(csv_path):
    os.remove(csv_path)

In [31]:
df.to_csv(csv_path, index=False)

In [38]:
csv_data=pd.read_csv("dataset.csv",encoding='utf-8')

In [39]:
csv_data.dropna()

,chunks,url
0,uttarkashi tunnel collapse ndrf team enters tu...,https://www.livemint.com/news/india/uttarkashi...
1,explore sign in epaper subscribe tuesday 2 jan...,https://www.livemint.com/news/india/uttarkashi...
2,top sections news india news world news econom...,https://www.livemint.com/news/india/uttarkashi...
3,a opinion markets stock markets commodity news...,https://www.livemint.com/news/india/uttarkashi...
4,multimedia collections videos webstories photo...,https://www.livemint.com/news/india/uttarkashi...
6,gainers losers top gainers top losers sun phar...,https://www.livemint.com/news/india/uttarkashi...
7,business news news india uttarkashi tunnel col...,https://www.livemint.com/news/india/uttarkashi...
8,a 15member ndrf team led by a commandant has b...,https://www.livemint.com/news/india/uttarkashi...
9,2 medical professionals have been stationed wi...,https://www.livemint.com/news/india/uttarkashi...
10,3 the dedicated freight corridor corporation o...,https://www.livemint.com/news/india/uttarkashi...


In [40]:
from langchain.document_loaders.csv_loader import CSVLoader

In [41]:
loader=CSVLoader(file_path=csv_path, source_column='chunks')

In [42]:
data=loader.load()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 2463: character maps to <undefined>